In [1]:
# 파일 경로 설정
file_path = '/Users/dryoon04/Documents/GitHub/schoolproject/vectorstore/open ai api key.txt'

# 파일 열기 (읽기 모드로 열기)
with open(file_path, 'r', encoding='utf-8') as file:
    # 파일 내용 읽어오기
    file_content = file.read()

In [2]:
import pandas as pd
import chromadb
import os
import openai


In [3]:
df = pd.read_csv('lyrics.csv')

In [4]:
client = chromadb.PersistentClient("chromadb/lyrics.db")
collection = client.get_or_create_collection(name="lyrics",metadata={"hnsw:space":"cosine"})


In [5]:
singers = []
song_titles = []
lyrics = []
ids = []
documents = []

for idx in range(len(df)):
    item = df.iloc[idx]
    singer = item['가수'].strip().lower()  # 가수 정보
    song_title = item['제목'].strip().lower()  # 노래 제목 정보
    lyric = item['가사'].strip().lower()  # 가사 정보

    # id = f"{singer}-{song_title}".replace(' ', '-')  # 고유한 ID 생성
    id = f"{singer}-{song_title}"
    document = f"{singer}:{song_title}:{lyric}"  # 문서 텍스트 생성
    
    
    ids.append(id)
    documents.append(document)

# DB 저장
collection.add(documents=documents,ids=ids)


Insert of existing embedding ID: stayc(스테이씨)-bubble
Insert of existing embedding ID: 영탁-폼미쳤다
Insert of existing embedding ID: 부석순(seventeen)-파이팅해야지(feat.이영지)
Insert of existing embedding ID: charlieputh-that'shilarious
Insert of existing embedding ID: 성시경-너의모든순간
Insert of existing embedding ID: nctdream-yogurtshake
Insert of existing embedding ID: 테이-monologue
Insert of existing embedding ID: charlieputh-that’snothowthisworks(feat.dan+shay)
Insert of existing embedding ID: 임영웅-이제나만믿어요
Insert of existing embedding ID: 김호중-인생은뷰티풀
Insert of existing embedding ID: 임영웅-우리들의블루스
Insert of existing embedding ID: 임영웅-abientot
Insert of existing embedding ID: 임영웅-인생찬가
Insert of existing embedding ID: 오마이걸(ohmygirl)-여름이들려(summercomes)
Insert of existing embedding ID: ive(아이브)-eitherway
Insert of existing embedding ID: 방탄소년단-butter
Insert of existing embedding ID: 정국-stillwithyou
Insert of existing embedding

In [6]:
user_query = input("검색어를 입력하세요: ")
vector_res = collection.query(query_texts=[f"{user_query}"],n_results=5)
print(user_query)
print(vector_res)


{'ids': [['charlieputh-that’snothowthisworks(feat.dan+shay)', '지민-likecrazy', "charlieputh-that'shilarious", 'newjeans-asap', 'fiftyfifty-cupid']], 'distances': [[0.8533503413200378, 0.8709683418273926, 0.8768119812011719, 0.8802019357681274, 0.8917070031166077]], 'metadatas': [[None, None, None, None, None]], 'embeddings': None, 'documents': [['charlieputh:that’snothowthisworks(feat.dan+shay):thought the day you disappeared\nthat it was over\ndidn\'t even hear you leavin\'\nsaw you with someone and\nthought that it was closure\nbut you still tell me\nthat you need me\nbaby, why, tell me, why\ncan\'t you just make up your mind?\nquit messin\' with my head\nyou can\'t say you hate me,\nthen call me when you\'re hurt\nbaby, you know that\'s not\nhow this works, no\nthat\'s not how this works\nyou can\'t walk away,\nthen come back to what we were\nbaby, you know that\'s not\nhow this works, no\nthat\'s not how this works, no\nno, no, no, no\noh, no, no, no, no\nbaby, that\'s not how t

In [7]:

os.environ["OPENAI_API_KEY"] = file_content
openai.api_key = os.getenv("OPENAI_API_KEY")
user_query = input("검색어를 입력하세요: ")
vector_res = collection.query(query_texts=[f"{user_query}"],n_results=5)
print(vector_res)
srchres = []
for v in vector_res['documents'][0]:
    item = v.split(':')
    srchres.append({"가수" : item[0].strip(),"제목" : item[1].strip(),"가사" : item[2].strip()})
    
completion = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
               {"role": "system","content": "You are an assistant who can answer questions about music lyricsBased on the following database In order to answer questions, you should only use facts from the databaseIf you feel you don't have enough information to answer a question, you should say, '답할수있는 정보가 없습니다.'."},
            {"role": "user", "content": f"{user_query}"},
            {"role": "assistant", "content": f"{srchres}"},
        ],
        temperature=1,
        max_tokens=256,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0,
    )
    
print(completion["choices"][0]["message"]["content"])


{'ids': [['지민-likecrazy', '정국-seven(feat.latto)-cleanver.', '정국-3d(feat.jackharlow)', 'v-lovemeagain', '디오(d.o.)-somebody']], 'distances': [[0.5546055436134338, 0.5575665272838066, 0.5663657784461975, 0.6221471428871155, 0.630937397480011]], 'metadatas': [[None, None, None, None, None]], 'embeddings': None, 'documents': [['지민:likecrazy:she’s saying\nbaby, 생각하지 마\nthere’s not a bad thing here tonight\nbaby, 떠나도 좋아\n있어 줘 오늘까지만\nwatch me go\n날 적셔 밤새도록 (away)\n아침도\n취해서 오지 않게\n시끄러운 음악 속에\n희미해진 나\n드라마 같은 뻔한 story\n익숙해져 가\n네가 알던 나를 찾기엔 멀리 온 걸까?\nyeah i know\nyou know\ni know\ni’d rather be\nlost in the lights\nlost in the lights\ni’m outta my mind\n이 밤의 끝을 잡아줘\n매일 밤\nyou spin me up high\n너를 품은 달\nlet me have a taste\ngive me a good ride\n(oh i’m fallin’)\nit’s gon’ be a good night\n(oh i’m fallin’)\nforever you and i\n거울 속에 비친 나\n하염없이 미쳐가\ni’m feelin’ so alive, wasting time\ni’d rather be\nlost in the lights\nlost in the lights\ni’m outta my mind\n이 밤의 끝을 잡아줘\n매일 밤\nyou spin me up

In [8]:
user_query = input("검색어를 입력하세요: ")
vector_res = collection.query(query_texts=[f"{user_query}"],n_results=10)
print(user_query)
print(vector_res)


{'ids': [['charlieputh-that’snothowthisworks(feat.dan+shay)', '지민-likecrazy', "charlieputh-that'shilarious", 'newjeans-asap', 'fiftyfifty-cupid', '정국,방탄소년단-dreamers[musicfromthefifaworldcupqatar2022officialsoundtrack](feat.fifasound)', 'akmu(악뮤)-lovelee', '방탄소년단-butter', 'paulblanco-summer(feat.be’o(비오))', 'lauv-stealtheshow(from“엘리멘탈”)']], 'distances': [[0.8533503413200378, 0.8709683418273926, 0.8768119812011719, 0.8802019357681274, 0.8917070031166077, 0.8919647192552597, 0.8931661248207092, 0.8956776261329651, 0.8999037742614746, 0.9000435749022401]], 'metadatas': [[None, None, None, None, None, None, None, None, None, None]], 'embeddings': None, 'documents': [['charlieputh:that’snothowthisworks(feat.dan+shay):thought the day you disappeared\nthat it was over\ndidn\'t even hear you leavin\'\nsaw you with someone and\nthought that it was closure\nbut you still tell me\nthat you need me\nbaby, why, tell me, why\ncan\'t you just make up your mind?\n